In [16]:
import numpy as np

# Read in and prepare text data

In [14]:
with open("goblet_book.txt", "r") as f:
    data = f.read()

book_chars = list(set(data))
K = len(book_chars)

In [15]:
charToInd = {c:i for i,c in enumerate(book_chars)}
indToChar = book_chars

In [ ]:
class RNN():
    def __init__(self, m=100, seed=123456789):
        np.random.seed(seed)

        self.m = m
        self.sigma = 0.01

        # Biases
        self.b = np.zeros(shape=(self.m,1))
        self.c = np.zeros(shape=(K,1))

        # Weights
        self.U = np.random.normal(shape=(self.m,K)) * self.sigma
        self.W = np.random.normal(shape=(self.m,m)) * self.sigma
        self.V = np.random.normal(shape=(self.K,m)) * self.sigma
    
    def synth(self, h0, x0, n):

        h = h0
        x = [x0]

        idxList = np.zeros(n)

        for i in range(n):
            a = self.W @ h + self.U @ x[-1] + self.b
            h = np.tanh(a)
            o = self.V @ h + self.c
            p = np.exp(o) / sum(np.exp(o))
            idx = np.random.choice(range(K),p=p)
            newX = np.zeros(K)
            newX[idx] = 1
            x.append(newX)
        
        return [np.argmax(c) for c in x]
            
        




In [17]:
np.exp(2)

7.38905609893065

In [ ]:
eta = 0.1
seq_length = 25